In [ ]:
import datetime 
import pandas as pd
import requests


In [ ]:
## Start Dates

start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 1)
delta = datetime.timedelta(days=31)


start_list =[]

while start_date <= end_date:
    start_list.append(start_date)
    start_date += delta
    
start_list

In [ ]:
## End Dates

start_date = datetime.date(2010, 2, 1)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days=31)


end_list =[]

while start_date <= end_date:
    end_list.append(start_date)
    start_date += delta
    
end_list

In [ ]:
#converting lists to strings 

end_str_list = []
for i in range(0,len(end_list)):
    end_str_list.append(end_list[i].strftime('%Y%m%d'))
    
start_str_list =[]
for i in range(0,len(start_list)):
    start_str_list.append(start_list[i].strftime('%Y%m%d'))
    
start_str_list

In [ ]:
#list of endpoints for API
complete_endpoints = []
for i in range(0,len(start_str_list)):
    complete_endpoints.append(f"{start_str_list[i]}:{end_str_list[i]}")

complete_endpoints

In [ ]:
# base url

base_url = 'https://www.ncdc.noaa.gov/swdiws/json/nx3tvs/'

# CELL_TYPE":"VARCHAR2","SHAPE":"MDSYS.SDO_GEOMETRY","MAX_SHEAR":"NUMBER","WSR_ID":"VARCHAR2","MXDV":"NUMBER","CELL_ID":"VARCHAR2","ZTIME":"VARCHAR2","AZIMUTH":"NUMBER","RANGE":"NUMBER

# empty lists
cell_type =[]
shape =[]
max_shear = []
wsr_id =[]
mxdv = []
cell_id =[]
ztime = []
azimuth = []
range_ = []

for dates in complete_endpoints:
    complete_url = base_url + dates
    print(complete_url)
    